In [ ]:

!pip install selenium
!pip install beautifulsoup4
!pip install cssutils
!pip install firebase-admin
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 24.7 MB/s 
     |████████████████████████████████| 140 kB 50.9 MB/s 
     |████████████████████████████████| 384 kB 48.2 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████████

In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from bs4 import BeautifulSoup
import sys
import time
import cssutils
import re
import datetime                            # Imports datetime library
import sqlite3 as lite

from os import remove

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--incognito")

In [ ]:

#if not firebase_admin._apps:
 # print(1)
  #cred = credentials.Certificate('firebase-sdk.json')
  #default_app = firebase_admin.initialize_app(cred, {
  #  'databaseURL' : 'https://apptraker-8c803-default-rtdb.firebaseio.com/'
  #})



cred = credentials.Certificate('firebase-sdk.json')
firebase_admin.initialize_app(cred,{
  'databaseURL':'https://apptraker-8c803-default-rtdb.firebaseio.com/'
})


FileNotFoundError: ignored

Scrapping Paginas dinamicas Padre

In [ ]:
class Scraping:
  def __init__(self, url, direccion, query, nombreBD):
    self.url = url
    self.direccion = direccion
    self.query = query
    self.driver = webdriver.Chrome('chromedriver',options=options)
    self.nombreBD = nombreBD
    self.soup =""
    self.Pagina()
    self.RecorrerPagina()
    self.Cargar_Pagina()

  def Pagina(self):
    self.driver.get(self.url+self.direccion+self.query)
    self.driver.maximize_window()

  def RecorrerPagina(self):
    iter=0
    while True:
      scrollHeight = self.driver.execute_script("return document.documentElement.scrollHeight")
      height=250*iter
      self.driver.execute_script("window.scrollTo(0, " + str(height) + ");")
      if height > scrollHeight:
          print('End of page')
          break
      time.sleep(0.5)
      iter+=1

  def Cargar_Pagina(self):
    body = self.driver.execute_script("return document.body")
    source = body.get_attribute('innerHTML')
    self.soup = BeautifulSoup(source, "html.parser")

Scrapping Hijo Lider

In [ ]:
class CrearDB:
  def __init__(self, db):
    self.db = lite.connect(db)



In [ ]:
class InteractuarDB(CrearDB):
  def __init__(self, db):
    CrearDB.__init__(self, db)
    self.cursor = self.db.cursor()
  def crearTable(self):
    tablas = [
              '''
          CREATE TABLE IF NOT EXISTS link_notebook(
            id integer primary key AUTOINCREMENT,
            link TEXT SQLITE_MAX_LENGTH,
            sku INTEGER UNIQUE,
            prod_img INTEGER
          )'''
    ]
    for tabla in tablas:
      self.cursor.execute(tabla)

  def incluirLinkDB(self, link, sku,prod_img):
    self.cursor.execute("""insert into link_notebook(link,sku,prod_img) VALUES (?,?,?)""", (link, sku,prod_img))
    self.db.commit()

  def printDB(self):
    print("entre")
    sentencia = "SELECT * FROM link_notebook;"
    self.cursor.execute(sentencia)
    #imprimir_link = self.cursor.fetchall()
    for x in self.cursor.fetchall():
      print(x)
    #imprimir_link = self.cursor.fetchone()
    #print(imprimir_link)
    #imprimir_link = self.cursor.fetchone()
    #print(imprimir_link)

  def estaEnDB(self, busqueda):
    db = lite.connect("link_lider.db")
    cursor = db.cursor()
    cursor.execute("SELECT * FROM link_notebook WHERE link=?", ("https://www.lider.cl/catalogo/product/sku/1258992/dell-notebook-inspiron-15-3502156intel-celeron4-gb-ram128-gb-ssdwin-10-hsilver",))
    print(cursor.fetchall())


In [ ]:
class Scraping_Lider(Scraping, InteractuarDB):
  def __init__(self, url, direccion, query, db):
    Scraping.__init__(self, url, direccion, query, db)
    InteractuarDB.__init__(self,db)

  def Buscar_Link_Scraping(self):
    link_completo = ""
    for producto in self.soup.find_all("li", 'ais-Hits-item'):
      link = producto.find("a", style="text-decoration: none; height: 100%; display: inline; background-color: white;")
      link_completo = str(self.url + link['href'])
      prod = producto.find("img", "img-hover-zoom img-fluid m-auto ")
      #print(link_completo)
      #print(prod)
      try:

        self.incluirLinkDB(link_completo, self.ExtraerSku(link['href']), str(prod['src']))
      except:
        pass
        #self.estaEnDB(link_completo)

  def ExtraerSku(self, link):
    varSplit = link.split('/')
    return str(varSplit[4])

  def Extraer_Descripcion(self):
    self.Cargar_Pagina()
    for descripcion in self.soup.find_all("tr", class_="MuiTableRow-root"):
      bandera=0
      array=[None, None]
      for contenido in descripcion.find_all("td", class_="MuiTableCell-root MuiTableCell-body"):
        array[bandera]=contenido.string
        bandera+=1
      if(array[0]=="Marca"):
        return (array[1])

  def Extraer_Precio(self):
    precio = self.soup.find("span", 'pdp-mobile-sales-price')
    try:
      precio = precio.string
      precio = precio.replace("$", "")
      precio = precio.replace(" ","")
      precio = int(precio.replace(".",""))
    except:
      precio=" "
    return (precio)

  def Extraer_Img(self):

    imagen = self.soup.find("li", "slide selected previous")
    #imagen = imagen.find("img", class_="img-fluid")
    #imagen = imagen['src']
    return imagen





In [ ]:
class Lider:
  def __init__(self):
    self.link_db = 'link_lider.db'
    self.url_lider = ""
    self.dir_lider_notebook = ""
    self.query_lider_notebook = ""
    self.inicio()
    self.Scraping_Link()

  def inicio(self):
    #con = lite.connect(':memory:')
    try:
      DB = InteractuarDB(self.link_db)
      DB.crearTable()
    except lite.OperationalError as error:
      print("Error al abrir:", error)

  def Scraping_Link(self):
    self.url_lider = "https://www.lider.cl"
    self.dir_lider_notebook = "/catalogo/category/Computación/Computadores/Notebooks"
    self.query_lider_notebook = "?page=1&sortBy=price_asc&hitsPerPage=200"
    self.Iniciar_Scraping()

  def Scraping_Productos(self):
    DB = lite.connect(self.link_db)
    cursor = DB.cursor()
    sentencia = "SELECT * FROM link_notebook;"
    cursor.execute(sentencia)
    ref = db.reference('Productos')
    #imprimir_link = self.cursor.fetchall()
    for link in cursor.fetchall():
      print(link)
      pagina_lider = Scraping_Lider(link[1], "", "", self.link_db)
      marca = pagina_lider.Extraer_Descripcion()
      precio = pagina_lider.Extraer_Precio()
      #imagen = pagina_lider.Extraer_Img()

      #print(imagen)
      #print(link[1])
      ref.push({'id' : str(link[0]),'marca':str(marca),'precio':str(precio),'link':str(link[1]), 'sku':str(link[2]),'imagen':str(link[3])})



  def Iniciar_Scraping(self):
    pagina_lider = Scraping_Lider(self.url_lider, self.dir_lider_notebook, self.query_lider_notebook, self.link_db)
    pagina_lider.Buscar_Link_Scraping()
    pagina_lider.printDB()
    #pagina_lider.estaEnDB("algo")
    #pagina_lider.Extraer_Marca()

    self.Scraping_Productos()

    #pagina_lider = Scraping_Lider(self.url_lider, self.dir_lider_notebook, self.query_lider_notebook, self.link_db)
    #pagina_lider.Extraer_Descripcion()
    #pagina_lider.Extraer_Precio()



In [ ]:
Lider()

